In [1]:
def convertir_a_centimos(precio_str):

    precio_str = precio_str.replace(",", ".")

    try:
        precio_float = float(precio_str)
    except ValueError:
        return None  
    precio_centimos = int(round(precio_float * 100))

    return precio_centimos


In [2]:
from flask import Flask, jsonify
import  pandas as pd
import joblib
app=Flask(__name__)
@app.route('/grafica/<Precio>/<VRAM>/<HDMI>/<Displayport>/<Energia>/<GDDR>')
def get_ordenador(Precio, VRAM, HDMI, Displayport,Energia, GDDR):
   
    try:
        gddr3 = gddr4 = gddr5 = gddr6 = gddr6x = gddr7 = False
        precioValor2 = convertir_a_centimos(Precio)
        if precioValor2 is None:
            return jsonify({"error": "Precio inválido. Debe ser un número válido."}), 400
        if VRAM.isdigit():
            vramValor = VRAM
        if HDMI.isdigit():
            hdmiValor = HDMI
        if Displayport.isdigit():
            displayValor = Displayport
        if Energia.isdigit():
            energiaValor = Energia
        match GDDR.upper().replace("GDDR", ""):
            case "3":
                gddr3 = True
            case "4":
                gddr4 = True
            case "5":
                gddr5 = True
            case "6":
                gddr6 = True
            case "6x":
                gddr6x = True
            case "7":
                gddr7 = True
            case _:
                return jsonify({"error": "Tipo de GDDR no válido"}), 400
        
        datos = pd.DataFrame([{
            "VRAM": vramValor,
            "HDMI": hdmiValor,
            "DisplayPort": displayValor,
            "Energia": energiaValor,
            "GDDR3": gddr3,
            "GDDR4": gddr4,
            "GDDR5": gddr5,
            "GDDR6": gddr6,
            "GDDR6X": gddr6x,
            "GDDR7": gddr7
        }])

        modelo = joblib.load('modelo_gb2.pkl')

        precio_predict =  modelo.predict(datos)

        if precio_predict[0]+(precio_predict[0]*0.15) < precioValor2:
            price_status =  "Muy mala, el precio está muy por encima del esperado."
        elif precio_predict[0]+(precio_predict[0]*0.15) > precioValor2 and precio_predict[0] < precioValor2:
            price_status = "Mala, el precio esta un poco por encima del esperado."
        elif precio_predict[0]-(precio_predict[0]*0.20) < precioValor2 and precio_predict[0] > precioValor2:
            price_status = "Buena, el precio esta por debajo del esperado."
        else: 
            price_status = "Muy buena, es un muy buen precio."
        return jsonify({
            "Precio_Predicho": round(precio_predict[0]),
            "Precio_Real": precioValor2,
            "Estado": price_status
        })
    except Exception as e:
        return jsonify({"error": f"Ocurrió un error inesperado: {str(e)}"}), 500
    

if __name__=="__main__":
    app.run()   # localhost at 5000

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2025 16:34:29] "GET /grafica/300/6/1/1/300/GDDR5 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:34:45] "GET /grafica/0/6/1/1/300/GDDR5 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:34:54] "GET /grafica/250/6/1/1/300/GDDR5 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:40:31] "GET /grafica/55,55/0/0/0/0/%20 HTTP/1.1" 400 -
127.0.0.1 - - [13/Mar/2025 16:41:45] "GET /grafica/333.00/4/3/3/300/GDDR3 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:42:10] "GET /grafica/333.00/4/1/1/300/GDDR3 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:42:19] "GET /grafica/333.00/4/1/1/300/GDDR7 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:43:32] "GET /grafica/333.00/4/1/1/300/GDDR7 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:43:43] "GET /grafica/300.00/4/1/1/300/GDDR7 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 16:53:02] "GET /grafica/300/0/0/0/0/%20 HTTP/1.1" 400 -
127.0.0.1 - - [13/Mar/2025 16:53:31] "GET /grafica/3333/0/0/0/0/%2